In [1]:
import re
import pandas as pd
from selenium import webdriver

In [2]:
browser = webdriver.Chrome('C:/Users/stepanovaev9/Downloads/chromedriver_win32/chromedriver.exe')
url = 'https://www.cbr.ru/currency_base/daily/'
browser.get(url)
browser.set_page_load_timeout(10)

In [3]:
days = range(1, 32)
monthes = range(1, 13)
years = range(2011, 2014)

In [6]:
currency_table = pd.DataFrame()
passed = []
for y in years:
    year = str(y)
    for m in monthes:
        month = str(m).zfill(2)
        for d in days:
            day = str(d).zfill(2)
            try:
                url = 'https://www.cbr.ru/currency_base/daily/?date_req={}.{}.{}'.format(day, month, year)
                browser.get(url)
                browser.set_page_load_timeout(5)
                
                header = browser.find_element_by_xpath('//*[@id="content"]/h2')
                date = re.findall('\d\d.\d\d.\d\d\d\d', header.text)[0]
    
                table_element = browser.find_elements_by_xpath('//*[@id="content"]/table')
                text = [row.text for row in table_element][0].split('\n')[1:]
                df = pd.DataFrame([[i[:3] for i in text],
                                   [i[4:7] for i in text],
                                   [i[-7:] for i in text]]).T
                
                df['Дата курса'] = date
                df['Дата'] = day+'.'+month+'.'+year
                currency_table = pd.concat([currency_table, df], axis=0)
                
            except:
                passed.append([day, month, year])
                
currency_table.columns = ['Числовой код валюты', 'Буквенный код валюты', 'Курс', 'Дата курса', 'Дата']
currency_table.drop_duplicates(inplace=True)

In [9]:
currency_table['Курс'] = currency_table['Курс'].str.replace(',', '.').apply(float)
currency_table['Дата курса'] = pd.to_datetime(currency_table['Дата курса'], dayfirst=True)

In [25]:
wrong_dates = []
for i in currency_table['Дата'].unique():
    try:
        i = pd.to_datetime(i, dayfirst=True)
    except:
        wrong_dates.append(i)

In [26]:
currency_table = currency_table[~currency_table['Дата'].isin(wrong_dates)]
currency_table['Дата'] = pd.to_datetime(currency_table['Дата'], dayfirst=True)

C:\Users\stepanovaev9\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
currency_table.to_excel('currencies.xlsx', index=False)

In [3]:
currency_table.head()

,Числовой код валюты,Буквенный код валюты,Курс,Дата курса,Дата
0,36,AUD,30.8938,2011-01-01,2011-01-01
1,944,AZN,37.9476,2011-01-01,2011-01-01
2,51,AMD,83.7833,2011-01-01,2011-01-01
3,974,BYR,10.1135,2011-01-01,2011-01-01
4,975,BGN,20.7015,2011-01-01,2011-01-01


In [4]:
currency_table.shape

(38374, 5)

In [7]:
currency_table['Дата'].min(), currency_table['Дата'].max()

(Timestamp('2011-01-01 00:00:00'), Timestamp('2013-12-31 00:00:00'))

In [5]:
currency_table['Буквенный код валюты'].nunique()

36